# Process AMOC and sea ice data to single time series

In [88]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import xarray as xr


In [ ]:
dir = '../../../Data/CESM104/'
areas = xr.open_dataset(dir + 'areacella_fx_CESM2-FV2_historical_r1i1p1f1_gn.nc')
sic = xr.open_dataset(dir + 'sic_mon_CESM104_abrupt2x_2500.nc')
moc = xr.open_dataset(dir + 'moc_ann_CESM104_abrupt2x_2500.nc')
moc

In [ ]:
moc_below500m = moc.where(moc.moc_z/100 > 500, drop=True)
# /100 to convert from cm to m
moc_above30N = moc_below500m.where(moc_below500m.lat_aux_grid > 30, drop=True)
moc_total = moc_above30N.sum(dim="moc_comp")
moc_total

In [97]:
maxmoc1 = moc_above30N.sel(moc_comp = 0).max(dim=("moc_z", "lat_aux_grid")) # Eulearian mean
maxmoc2 = moc_above30N.sel(moc_comp = 1).max(dim=("moc_z", "lat_aux_grid")) # Eddy-Induced (bolus)
maxmoc3 = moc_above30N.sel(moc_comp = 2).max(dim=("moc_z", "lat_aux_grid")) # submeso
maxmoc = moc_total.max(dim=("moc_z", "lat_aux_grid")) 

In [ ]:
moc_to_save = maxmoc.isel(transport_reg = 1)
#moc_to_save.to_netcdf('../../Other_data/longrunmip_data/max_moc_CESM104_abrupt2x.nc')

In [ ]:
fig, ax = plt.subplots(figsize  = (12,5))
maxmoc.moc.isel(transport_reg = 0).plot()
maxmoc.moc.isel(transport_reg = 1).plot()
ax.grid()

In [127]:
NH_seaicearea = (sic * areas.areacella).where(sic.lat>0, drop=True).sum(dim = ("lat", "lon"))
SH_seaicearea = (sic * areas.areacella).where(sic.lat<0, drop=True).sum(dim = ("lat", "lon"))
NH_seaicearea = NH_seaicearea.rename({"sic": "seaicearea"})
SH_seaicearea = SH_seaicearea.rename({"sic": "seaicearea"})


In [129]:
NHseaice = NH_seaicearea.groupby('time.year').mean('time')
SHseaice = SH_seaicearea.groupby('time.year').mean('time')

#NHseaice.to_netcdf('../../Other_data/longrunmip_data/NH_seaice.nc')
#SHseaice.to_netcdf('../../Other_data/longrunmip_data/SH_seaice.nc')

In [ ]:
(NHseaice.seaicearea/10**12).plot()
(SHseaice.seaicearea/10**12).plot()
